In [28]:
import numpy as np
import os
import pandas as pd
import tensorflow.compat.v1 as tf 
import matplotlib.pyplot as plt
import time
from tensorflow.compat.v1 import keras
from tensorflow.compat.v1.keras import losses,optimizers
import cv2
from tensorflow.compat.v1.keras import layers,metrics
from tensorflow.compat.v1.keras.applications.resnet50 import ResNet50
tf.enable_eager_execution() 


In [17]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150
def get_training_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [18]:
train = get_training_data('.\\chest_xray\\train')
test = get_training_data('.\\chest_xray\\test')
val = get_training_data('.\\chest_xray\\val')

In [19]:
x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)
    
for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

In [20]:
x_train = np.array(x_train)# / 255
x_val = np.array(x_val)# / 255
x_test = np.array(x_test)# / 255

In [21]:
x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [29]:
resnet = ResNet50(include_top=False,weights=None,input_shape=(img_size,img_size,1),pooling="avg")
model=keras.Sequential()
model.add(resnet)
model.add(layers.Dense(units=1000,activation="relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(units=1,activation="sigmoid"))
model.summary()

model.compile(loss="binary_crossentropy",optimizer="Adam",metrics=['accuracy'])
#model.load_weights('./weights/resnetweights')


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23581440  
_________________________________________________________________
dense_12 (Dense)             (None, 1000)              2049000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 1001      
Total params: 25,631,441
Trainable params: 25,578,321
Non-trainable params: 53,120
_________________________________________________________________


In [50]:

history = model.fit(x=x_train,y=y_train,batch_size=32,epochs=5,validation_data=(x_val,y_val),shuffle=True,)
model.save_weights('./weights/resnetweights')

Train on 5216 samples, validate on 16 samples
Epoch 1/5
5216/5216 [==============================] - 18s 3ms/sample - loss: 0.0470 - acc: 0.9850 - val_loss: 0.1487 - val_acc: 0.8750
Epoch 2/5
5216/5216 [==============================] - 18s 3ms/sample - loss: 0.0412 - acc: 0.9891 - val_loss: 6.6377 - val_acc: 0.5000
Epoch 3/5
5216/5216 [==============================] - 18s 3ms/sample - loss: 0.0120 - acc: 0.9973 - val_loss: 19.8715 - val_acc: 0.5000
Epoch 4/5
5216/5216 [==============================] - 18s 3ms/sample - loss: 0.0091 - acc: 0.9971 - val_loss: 1.8118 - val_acc: 0.6875
Epoch 5/5
5216/5216 [==============================] - 18s 3ms/sample - loss: 9.6753e-04 - acc: 0.9996 - val_loss: 6.4926e-04 - val_acc: 1.0000


In [51]:
test = model.evaluate(x_test,y_test)

624/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================